# Greedy search
On each step found the world with lowest perplexity\
(+) faster than brute force\
(-) can be found local optimum, not global optimum, because if the world best on this step that it's not guarantee that the combination of n worlds is the best

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import numpy as np
import random
import time

In [2]:
df = pd.read_csv('/kaggle/input/santa-2024/sample_submission.csv')

In [3]:
df

,id,text
0,0,advent chimney elf family fireplace gingerbrea...
1,1,advent chimney elf family fireplace gingerbrea...
2,2,yuletide decorations gifts cheer holiday carol...
3,3,yuletide decorations gifts cheer holiday carol...
4,4,hohoho candle poinsettia snowglobe peppermint ...
5,5,advent chimney elf family fireplace gingerbrea...


In [4]:
from metric import PerplexityCalculator
scorer = PerplexityCalculator('/kaggle/input/gemma-2/transformers/gemma-2-9b/2')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
start_time = time.time()
for idx, text in enumerate(df["text"].to_list()):
    words = text.split()
    greedy_text = ""
    
    # Iteratively add the word resulting in minimum perplexity
    while len(words):
        input_texts = [(greedy_text + " " + word).strip() for word in words]
        perplexities = scorer.get_perplexity(input_texts)
        greedy_text = (greedy_text + " " + words.pop(np.argmin(perplexities))).strip()
    
    print(f"Sample {idx}: perplexity={perplexities[0]:.2f}")
    print(greedy_text)
    df.loc[idx, "greedy_text"] = greedy_text
    elapsed_time = time.time() - start_time
    print(f"Time elapsed for Sample {idx}: {elapsed_time:.2f} seconds")

Sample 0: perplexity=1327.97
scrooge mistletoe ornament family advent fireplace chimney elf reindeer gingerbread
Time elapsed for Sample 0: 6.66 seconds
Sample 1: perplexity=2017.02
scrooge mistletoe ornament and reindeer family advent fireplace chimney elf night sleep gingerbread bake the walk laugh give jump drive
Time elapsed for Sample 1: 29.26 seconds
Sample 2: perplexity=967.78
yuletide gifts grinch ornament nutcracker decorations holiday stocking holly jingle sleigh carol cheer chimney naughty nice beard workshop polar magi
Time elapsed for Sample 2: 51.81 seconds
Sample 3: perplexity=762.59
yuletide gifts unwrap holiday cheer the nutcracker and grinch decorations ornament stocking holly jingle sleigh carol sing cheer chimney visit naughty nice beard eat relax polar workshop of is magi
Time elapsed for Sample 3: 103.10 seconds
Sample 4: perplexity=283.85
eggnog fruitcake poinsettia snowglobe wreath candle cookie star candy peppermint chocolate milk hohoho merry and joy peace hop

In [8]:
submission = pd.DataFrame(columns=['id', 'text'])

for idx, row in df.iterrows():

    id = row.id
    optimized_text = row.greedy_text

    submission.loc[idx] = {
        'id': id,
        'text': optimized_text
    }

submission.to_csv("submission_greedy.csv", index=False)

print('submission created')

submission created
